In [1]:
# Changed back to the RFM scoring and using the old method
# Fill the lapsed with "L" group
import pandas as pd
import numpy as np
import os
import datetime
import logging
import hashlib
import gc
import glob
logging.basicConfig(filename='CRM_newscore_2019Q2.log', level=logging.INFO)
logging.info('Started')

samplerows = None

lastdate = datetime.date(2019,3,30) # Recent Saturday
active_Sunday = str(lastdate-datetime.timedelta(days=52*7-1))
lapsed_Sunday = str(lastdate-datetime.timedelta(days=52*7*4-1))
Beginning_18_months_ago=str(lastdate-datetime.timedelta(days=52*7*1.5-1))
lastdate=str(lastdate)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Scoring Start on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=


folder_write = '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_'+str(datetime.datetime.now().date())+'/'
try:
    os.stat(folder_write)
except:
    os.mkdir(folder_write)
    
# Adding control members

Lapsed Start on: 2015-04-05
Active Start on: 2018-04-01
Scoring Start on: 2017-10-01
Last Saturday: 2019-03-30


In [2]:
#To check the 1st date

chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_time',
                   'transaction_id','location_id','total_transaction_amt']): #Add back the transaction info
    chunk['total_transaction_amt']=chunk['total_transaction_amt'].astype(float)
    chunk = chunk.drop_duplicates()
    
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
    

'''
dftrans_before_20180922 = pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_0922/combinedtransactions_0922.csv',dtype=str)
#dftrans = dftrans[dftrans['transaction_date']>=lapsed]
dftrans_before_20180922['total_transaction_amt']=dftrans_before_20180922['total_transaction_amt'].astype(float)
dftrans_before_20180922['total_transaction_units']=dftrans_before_20180922['total_transaction_units'].astype(float)

dftrans_before_20180922 = dftrans_before_20180922[['customer_id_hashed','transaction_date','transaction_time',
                   'transaction_id','location_id','total_transaction_units',
                   'total_transaction_amt']].drop_duplicates()
'''

del chunk
print("Earliest Date:" + str(dftrans_before_20180922['transaction_date'].min()))

dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()

print("Deduped",datetime.datetime.now())


logging.info("Deduped: "+str(datetime.datetime.now()))
del dftrans_before_20180922['transaction_time']

gc.collect()


1 2019-04-12 10:45:24.459670
2 2019-04-12 10:46:17.315953
3 2019-04-12 10:47:02.558216
4 2019-04-12 10:48:05.879345
5 2019-04-12 10:49:14.009180
6 2019-04-12 10:50:02.740514
7 2019-04-12 10:50:46.684698
8 2019-04-12 10:51:30.171693
9 2019-04-12 10:52:14.589230
10 2019-04-12 10:53:05.927263
11 2019-04-12 10:54:01.709333
12 2019-04-12 10:55:04.356556
13 2019-04-12 10:56:08.963409
14 2019-04-12 10:57:23.544976
15 2019-04-12 10:58:22.686567
Earliest Date:2016-06-26
Deduped 2019-04-12 11:18:30.216930


84

In [5]:
def recrusive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
# Up to 2019-03-30 
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recrusive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,3,31)]
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)]
print(historical_daily_df.shape)
print("Min_Date: "+str(historical_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(historical_daily_df['week_end_dt'].max()))

(20, 2)
Min_Date: 2018-09-29
Max_Date: 2019-02-09


In [6]:
new_daily_data_folder="/home/jian/BigLots/2019_by_weeks/"
new_daily_data_list=list(recrusive_file_gen(new_daily_data_folder))
new_daily_data_list=[x for x in new_daily_data_list if (".txt" in x) & ("DailySales" in x)]
new_daily_data_list=[x for x in new_daily_data_list if "hist" not in x]

new_daily_df=pd.DataFrame({"file_path":new_daily_data_list})

new_daily_df['week_end_dt']=new_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("2019_by_weeks/MediaStorm_")[1][:10])
new_daily_df['week_end_dt']=new_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
new_daily_df=new_daily_df[new_daily_df['week_end_dt']>historical_daily_df['week_end_dt'].max()]
new_daily_df=new_daily_df[new_daily_df['week_end_dt']<=datetime.date(2019,3,31)]
print(new_daily_df.shape)
print("Min_Date: "+str(new_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(new_daily_df['week_end_dt'].max()))

daily_df_file_after_20180922=historical_daily_df.append(new_daily_df)
new_dailysales_files=daily_df_file_after_20180922['file_path'].tolist()

(7, 2)
Min_Date: 2019-02-16
Max_Date: 2019-03-30


In [7]:
combined_rewards_transaction_after_20180922_agg=pd.DataFrame() 
count_i=1
print("Total Files: "+str(len(new_dailysales_files)))
for file_daily in new_dailysales_files:
    df=pd.read_table(file_daily,nrows = None,sep= '|',dtype =str,usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df.groupby(['customer_id_hashed','transaction_dt','transaction_id','location_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    # df=df.drop_duplicates()

    
    combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.append(df)
    print(count_i,"done",datetime.datetime.now())
    count_i+=1
del df
gc.collect()
combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.rename(columns={"transaction_dt":"transaction_date"})


Total Files: 27
1 done 2019-04-12 11:20:12.607483
2 done 2019-04-12 11:21:05.715433
3 done 2019-04-12 11:21:46.644834
4 done 2019-04-12 11:22:29.117383
5 done 2019-04-12 11:23:14.552269
6 done 2019-04-12 11:24:02.240329
7 done 2019-04-12 11:24:53.153709
8 done 2019-04-12 11:25:52.811013
9 done 2019-04-12 11:27:14.066929
10 done 2019-04-12 11:28:34.582356
11 done 2019-04-12 11:30:21.403691
12 done 2019-04-12 11:32:03.484094
13 done 2019-04-12 11:34:12.357863
14 done 2019-04-12 11:35:30.620907
15 done 2019-04-12 11:36:30.473861
16 done 2019-04-12 11:37:24.836252
17 done 2019-04-12 11:38:27.325814
18 done 2019-04-12 11:39:39.233348
19 done 2019-04-12 11:40:38.345814
20 done 2019-04-12 11:41:38.082813
21 done 2019-04-12 11:42:47.101810
22 done 2019-04-12 11:43:57.876579
23 done 2019-04-12 11:45:12.994440
24 done 2019-04-12 11:46:23.356492
25 done 2019-04-12 11:47:29.710666
26 done 2019-04-12 11:48:38.063287
27 done 2019-04-12 11:49:46.923144


In [8]:
gc.collect()

combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.groupby(['customer_id_hashed','transaction_date','transaction_id','location_id'])['item_transaction_amt'].sum().to_frame().reset_index()
combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.rename(columns={"item_transaction_amt":"total_transaction_amt"})

all_rewards_since_201606=dftrans_before_20180922.append(combined_rewards_transaction_after_20180922_agg)

del dftrans_before_20180922
del combined_rewards_transaction_after_20180922_agg
gc.collect()

42

In [9]:
all_rewards_since_201606.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/BL_Rewards_Transactions_18_months_to_20190330_JL_"+str(datetime.datetime.now().date())+".csv")
gc.collect()

0

In [10]:
all_rewards_since_201606.head(2)

,customer_id_hashed,transaction_date,transaction_id,location_id,total_transaction_amt
74861588,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,2017-01-19,6044,1292,32.8
148150940,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,1317,4061,58.7


In [11]:

'''
# Lapsed data

lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str) # Doesn't go to score at all, so no need to read all columns

print("Lapsed earliest:" + lapsed['transaction_date'].min())
print("Lapsed newest:" + lapsed['transaction_date'].max())
'''



'\n# Lapsed data\n\nlapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",\n                     sep=",",usecols=[\'customer_id_hashed\',\'transaction_date\'],dtype=str) # Doesn\'t go to score at all, so no need to read all columns\n\nprint("Lapsed earliest:" + lapsed[\'transaction_date\'].min())\nprint("Lapsed newest:" + lapsed[\'transaction_date\'].max())\n'

In [12]:
#Getting the store for an id

frequently_visit_stores_18_months=all_rewards_since_201606[all_rewards_since_201606['transaction_date']>=Beginning_18_months_ago]

frequently_visit_stores_2=frequently_visit_stores_18_months.groupby(['customer_id_hashed','location_id'])['total_transaction_amt'].sum().to_frame().reset_index().rename(columns={"total_transaction_amt":"sales"})
frequently_visit_stores_18_months=frequently_visit_stores_18_months.groupby(['customer_id_hashed','location_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})

frequently_visit_stores_18_months=pd.merge(frequently_visit_stores_18_months,frequently_visit_stores_2,on=['customer_id_hashed','location_id'],how="outer")
del frequently_visit_stores_2
print(frequently_visit_stores_18_months.shape)
frequently_visit_stores_18_months=frequently_visit_stores_18_months.sort_values(['customer_id_hashed','trans','sales'],ascending=[True,False,False])

frequently_visit_stores_18_months=frequently_visit_stores_18_months[['customer_id_hashed','location_id']].drop_duplicates("customer_id_hashed")
print(frequently_visit_stores_18_months.shape)
frequently_visit_stores_18_months.to_csv(folder_write+"frequently_visit_stores_18_months.csv",index=False)

(24905063, 4)
(20569010, 2)


In [13]:
# Read the registeration master file to fetch the location_id when registers if no transactions recorded
# To be added
# Below

In [14]:
gc.collect()

80

In [15]:
###get recency
dfrecency=all_rewards_since_201606[['customer_id_hashed','transaction_date']].sort_values("transaction_date",ascending=False).drop_duplicates()#Allready combined

print (min(dfrecency['transaction_date']))
print (max(dfrecency['transaction_date']))
dfrecency = dfrecency.drop_duplicates('customer_id_hashed')
dfrecency.to_csv(folder_write + 'dfrecency.csv',index = False)

2016-06-26
2019-03-30


In [16]:
dfrecency['transaction_date'] = pd.to_datetime(dfrecency['transaction_date'])
dfrecency['recency'] =  datetime.datetime.strptime(str(lastdate), '%Y-%m-%d').date() - dfrecency['transaction_date']
dfrecency['recency'] = dfrecency['recency'].apply(lambda x:x.days)
dfrecency['recency'] = np.ceil((dfrecency['recency']+1)/30)

dfrecency = dfrecency[['customer_id_hashed','recency']]
dfrecency = dfrecency.drop_duplicates('customer_id_hashed')
dfrecency.to_csv(folder_write + 'dfrecency2.csv',index = False)

dfrecency.shape

(25770895, 2)

In [17]:
dfiddetail = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',nrows = samplerows)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')
#####
new_sign_ups_2019_list=list(recrusive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
new_sign_ups_2019_list=sorted([x for x in new_sign_ups_2019_list if "ster" in x])
for file_nnew_signups in new_sign_ups_2019_list:
    df=pd.read_table(file_nnew_signups,dtype=str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'],sep="|")
    dfiddetail=df.append(dfiddetail) # Already sorted and newest kept on the top
    print(datetime.datetime.now(),file_nnew_signups)
dfiddetail=dfiddetail.drop_duplicates("customer_id_hashed")

######
dfiddetail2 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     nrows = samplerows,dtype = 'str',sep = '|',
                       usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail = dfiddetail.append(dfiddetail2,ignore_index = True)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')
dfiddetail = dfiddetail.drop_duplicates('email_address_hash')

del dfiddetail2
del df
gc.collect()

2019-04-12 12:37:28.651520 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-12/MediaStormMasterBiWeekly20190115-132855-055.txt
2019-04-12 12:37:34.832000 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-26/MediaStormMasterBiWeekly20190129-130902-016.txt
2019-04-12 12:37:40.593709 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-02/MediaStormMasterWeekly20190205-111610-675.txt
2019-04-12 12:37:46.546554 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-09/MediaStormMasterWeekly20190212-122428-267.txt
2019-04-12 12:37:52.081384 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormMasterWeekly20190219-113650-867.txt
2019-04-12 12:37:57.978011 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormMasterWeekly20190226-112921-061.txt
2019-04-12 12:38:03.707337 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormMasterWeekly20190305-112945-302.txt
2019-04-12 12:38:09.461360 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormMasterWe

150

In [18]:
all_rewards_since_201606['transactions'] = 1
dftotal = all_rewards_since_201606[['customer_id_hashed','total_transaction_amt','transactions']].groupby(['customer_id_hashed']).sum().reset_index().rename(columns={"total_transaction_amt":"sales"})

dftotal = pd.merge(dftotal,dfrecency,on = 'customer_id_hashed',how='outer')
del dfrecency

gc.collect()

42

In [19]:
dftotal = dftotal.sort_values(['transactions','recency','sales'],ascending = [0,1,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'Transindex'})

dftotal = dftotal.sort_values(['sales','recency','transactions'],ascending = [0,1,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'Amtindex'})

dftotal = dftotal.sort_values(['recency','transactions','sales'],ascending = [1,0,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'recencyindex'})

c_ids = len(dftotal.index)
logging.info('total customers from transaction and amt: ')
logging.info(c_ids)
c_ids = np.ceil(c_ids/5.0)

dftotal['Transindex'] = np.ceil((dftotal['Transindex']+1)/c_ids)
dftotal['Amtindex'] = np.ceil((dftotal['Amtindex']+1)/c_ids)
dftotal['recencyindex'] = np.ceil((dftotal['recencyindex']+1)/c_ids)

dftotal['RFM'] = dftotal['recencyindex']*100 + dftotal['Transindex']*10 + dftotal['Amtindex']
'''
dftotal = dftotal.sort_values(['RFM','recency','transactions',
                               'sales'],ascending = [1,1,0,0])


dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'frmindex'})
c_ids = len(dftotal.index)
c_ids = np.ceil(c_ids/10.0)
dftotal['frmindex'] = np.ceil((dftotal['frmindex']+1)/c_ids)

dftotal.to_csv(folder_write + 'dfrfm.csv',index = False)


# In[13]:


dftotal = pd.read_csv(folder_write + 'dfrfm.csv')
'''

# In[14]:


dftotal = dftotal[['customer_id_hashed','RFM','recency','transactions','sales']]
gc.collect()

135

In [20]:
# In[15]:
dfrecency = pd.read_csv(folder_write + 'dfrecency.csv')
dfrecency['active'] = np.where(dfrecency['transaction_date']>=active_Sunday,'active',
                    np.where(dfrecency['transaction_date']>=lapsed_Sunday,'lapsed','other'))
print(dfrecency['active'].unique())


# In[16]:


dftotal = pd.merge(dftotal,dfrecency[['customer_id_hashed','active']],on = 'customer_id_hashed')


# In[17]:


dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].astype('str')
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].str[0:5]
dfiddetail['customer_zip_code'].fillna('00000',inplace = True)
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].apply(lambda x:x.zfill(5))
print(len(dfiddetail.index))


# In[18]:


print("totalids_trans:",len(dftotal.index))
dftotal = pd.merge(dftotal,dfiddetail,on = 'customer_id_hashed') #drop_duplicates with inner merge
print("totalids_trans_mergewithmaster:",len(dftotal.index))




['active' 'lapsed']
25083173
totalids_trans: 25770895
totalids_trans_mergewithmaster: 24077565


In [21]:
dftotal.head(2)

,customer_id_hashed,RFM,recency,transactions,sales,active,email_address_hash,customer_zip_code
0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,111.0,1.0,1979,31812.53,active,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607
1,14b78c9088ecdbde7a771c49664157cfe588f86ae5ce5c...,111.0,1.0,1919,42819.02,active,d72cf03e94da3a0f89abc29df597ff699420ab1c2671b3...,87501


In [28]:
dftotal = dftotal.sort_values(['RFM','recency','transactions',
                               'sales'],ascending = [1,1,0,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'frmindex'})
c_ids = len(dftotal.index)
c_ids = np.ceil(c_ids/10.0)
dftotal['frmindex'] = np.ceil((dftotal['frmindex']+1)/c_ids)


zipmap = pd.read_csv('/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/zip_with_ta_dma.csv',dtype = 'str')
zipmap['zipcodegroup'] = zipmap['revenue_flag']
zipmap = zipmap[['zip','zipcodegroup']].drop_duplicates('zip')
zipmap.columns = ['customer_zip_code','zipcodegroup']
dftotal = pd.merge(dftotal,zipmap,on ='customer_zip_code',how = 'left' )
print(dftotal['zipcodegroup'].unique())
dftotal['zipcodegroup'].fillna('T',inplace = True)
print(dftotal['zipcodegroup'].unique())


# In[21]:

dftotal.to_csv(folder_write + 'dfrfm_wemail.csv',index = False)
print("Final wemailcsv:",dftotal.shape)

['P' 'S' nan 'T']
['P' 'S' 'T']
Final wemailcsv: (24077565, 10)


In [29]:
# Walking Death before the learliest date in customer transaction data as 2016-06-26

# Getting the primary stores for each member

In [30]:
# frequently_visit_stores_18_months=pd.read_csv(folder_write+"frequently_visit_stores_18_months.csv",dtype=str)
register_stores=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/output_sing_up_location/BL_id_by_register_store_JL_2019-04-09.csv",dtype=str)
register_stores=register_stores[['customer_id_hashed','sign_up_location']].rename(columns={"sign_up_location":"location_id"})




In [31]:
store_for_ids=frequently_visit_stores_18_months.append(register_stores)
store_for_ids=store_for_ids.drop_duplicates("customer_id_hashed")
store_for_ids.shape

(26754444, 2)

In [32]:
del frequently_visit_stores_18_months
del register_stores
gc.collect()

139

In [33]:
dftotal=pd.merge(dftotal,store_for_ids,on="customer_id_hashed",how="left")

In [34]:
# read the quadrant by store for 2018 Q4

Q4_store_quadrant=pd.read_excel("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Excel_BL_2018_Q4_post_YoY_small_JL_2019-03-04.xlsx",
                                dtype=str,sheetname="Q4_Store_Quadrant_Defination",usecols=['location_id','Quadrant'])
print(Q4_store_quadrant.shape)
print(len(Q4_store_quadrant['location_id'].unique()))

dftotal=pd.merge(dftotal,Q4_store_quadrant,on="location_id",how="left")
dftotal['Quadrant']=dftotal['Quadrant'].fillna("Quadrant III")

dftotal.head(2)

(1361, 2)
1361


,frmindex,customer_id_hashed,RFM,recency,transactions,sales,active,email_address_hash,customer_zip_code,zipcodegroup,location_id,Quadrant
0,1.0,b78dccfd64ce659af7061c5743b6b9305814336bc9ddec...,111.0,1.0,1979,31812.53,active,4836bfc632a2a5987d47fa6bf0637c21c4466a0e99ddee...,28607,P,1684,Quadrant I
1,1.0,14b78c9088ecdbde7a771c49664157cfe588f86ae5ce5c...,111.0,1.0,1919,42819.02,active,d72cf03e94da3a0f89abc29df597ff699420ab1c2671b3...,87501,P,4147,Quadrant IV


In [35]:
gc.collect()

89437

# Summary

In [36]:
dftotal['frmindex']=dftotal['frmindex'].apply(lambda x: str(int(float(x))).zfill(2))
dftotal['customer_zip_code']=dftotal['customer_zip_code'].apply(lambda x: x.zfill(5))
dftotal['frmindex']=dftotal['frmindex'].apply(lambda x:"D"+x)


In [37]:
dftotal.to_csv(folder_write + 'dfrfm_final_details_wemail_zip_StoreQuad.csv',index = False)
gc.collect()

0

In [38]:
# Random 500000 ids as control

In [39]:
dftotal['frmindex'].unique().tolist()

['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10']

In [40]:
dftotal['HML_Group']=np.where(dftotal['frmindex'].isin(['D01','D02','D03','D04']),"H",
                                        np.where(dftotal['frmindex'].isin(['D05','D06','D07']),"M","L"))

dftotal['segment_2019Q2']=dftotal['Quadrant']+"_"+dftotal['zipcodegroup']+"_"+dftotal['HML_Group']+"_2019Q2"


In [42]:
import random
random.seed(1)
total_rows=len(dftotal)

test_all_df=pd.DataFrame()
control_all_df=pd.DataFrame()

i_counter=0

for seg,group in dftotal.groupby(['segment_2019Q2']):
    random_list=random.sample(range(len(group)), int(np.round(len(group)/total_rows*500000)))

    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    df_control=group[group['index'].isin(random_list)]
    df_test=group[~group['index'].isin(random_list)]
    
    df_control['segment_2019Q2']="C_"+df_control['segment_2019Q2']
    df_test['segment_2019Q2']="T_"+df_test['segment_2019Q2']
    test_all_df=test_all_df.append(df_test)
    control_all_df=control_all_df.append(df_control)
    i_counter+=1
    print(i_counter,datetime.datetime.now())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 2019-04-12 13:30:49.479493
2 2019-04-12 13:31:03.016016
3 2019-04-12 13:31:19.856100
4 2019-04-12 13:31:38.489731
5 2019-04-12 13:31:56.575115
6 2019-04-12 13:32:16.333787
7 2019-04-12 13:32:36.410781
8 2019-04-12 13:32:57.330702
9 2019-04-12 13:33:20.367770
10 2019-04-12 13:33:46.178435
11 2019-04-12 13:34:17.234605
12 2019-04-12 13:34:49.034214
13 2019-04-12 13:35:23.394224
14 2019-04-12 13:35:57.082590
15 2019-04-12 13:36:33.013831
16 2019-04-12 13:37:08.212425
17 2019-04-12 13:37:43.792461
18 2019-04-12 13:38:18.709902
19 2019-04-12 13:39:00.010782
20 2019-04-12 13:39:45.935868
21 2019-04-12 13:40:35.855834
22 2019-04-12 13:41:27.110888
23 2019-04-12 13:42:18.877307
24 2019-04-12 13:43:11.959976
25 2019-04-12 13:44:05.470911
26 2019-04-12 13:44:59.725558
27 2019-04-12 13:45:54.836471
28 2019-04-12 13:47:08.922312
29 2019-04-12 13:48:38.518706
30 2019-04-12 13:50:18.444962
31 2019-04-12 13:52:03.269632
32 2019-04-12 13:53:47.641249
33 2019-04-12 13:55:37.005274
34 2019-04-12 13:57

In [43]:
del dftotal
gc.collect()

37

In [44]:
test_all_df.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_2019-04-12/all_test.csv",index=False)
control_all_df.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_2019-04-12/all_control.csv",index=False)

folder_write_inner = '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_2019-04-12/by_group/'
try:
    os.stat(folder_write_inner)
except:
    os.mkdir(folder_write_inner)

In [45]:
i_counter=0
for seg,group in test_all_df.groupby(['segment_2019Q2']):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q2']].rename(columns={"segment_2019Q2":"segment"})
    group.to_csv(folder_write_inner+seg+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())

1 T_Quadrant III_P_H_2019Q2 2019-04-12 14:08:35.899690
2 T_Quadrant III_P_L_2019Q2 2019-04-12 14:08:47.375873
3 T_Quadrant III_P_M_2019Q2 2019-04-12 14:08:54.426114
4 T_Quadrant III_S_H_2019Q2 2019-04-12 14:08:56.575794
5 T_Quadrant III_S_L_2019Q2 2019-04-12 14:08:59.348704
6 T_Quadrant III_S_M_2019Q2 2019-04-12 14:09:01.183223
7 T_Quadrant III_T_H_2019Q2 2019-04-12 14:09:02.117560
8 T_Quadrant III_T_L_2019Q2 2019-04-12 14:09:04.391562
9 T_Quadrant III_T_M_2019Q2 2019-04-12 14:09:05.410305
10 T_Quadrant II_P_H_2019Q2 2019-04-12 14:09:13.085480
11 T_Quadrant II_P_L_2019Q2 2019-04-12 14:09:17.976533
12 T_Quadrant II_P_M_2019Q2 2019-04-12 14:09:23.586020
13 T_Quadrant II_S_H_2019Q2 2019-04-12 14:09:25.251105
14 T_Quadrant II_S_L_2019Q2 2019-04-12 14:09:26.268041
15 T_Quadrant II_S_M_2019Q2 2019-04-12 14:09:27.470851
16 T_Quadrant II_T_H_2019Q2 2019-04-12 14:09:28.203779
17 T_Quadrant II_T_L_2019Q2 2019-04-12 14:09:28.586831
18 T_Quadrant II_T_M_2019Q2 2019-04-12 14:09:29.208656
19 T_Quadr

In [46]:
i_counter=0
for seg,group in control_all_df.groupby(['segment_2019Q2']):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q2']].rename(columns={"segment_2019Q2":"segment"})
    group.to_csv(folder_write_inner+seg+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())

1 C_Quadrant III_P_H_2019Q2 2019-04-12 14:11:32.243993
2 C_Quadrant III_P_L_2019Q2 2019-04-12 14:11:32.457931
3 C_Quadrant III_P_M_2019Q2 2019-04-12 14:11:32.583040
4 C_Quadrant III_S_H_2019Q2 2019-04-12 14:11:32.622672
5 C_Quadrant III_S_L_2019Q2 2019-04-12 14:11:32.677328
6 C_Quadrant III_S_M_2019Q2 2019-04-12 14:11:32.711672
7 C_Quadrant III_T_H_2019Q2 2019-04-12 14:11:32.730265
8 C_Quadrant III_T_L_2019Q2 2019-04-12 14:11:32.773138
9 C_Quadrant III_T_M_2019Q2 2019-04-12 14:11:32.793373
10 C_Quadrant II_P_H_2019Q2 2019-04-12 14:11:32.937548
11 C_Quadrant II_P_L_2019Q2 2019-04-12 14:11:33.026331
12 C_Quadrant II_P_M_2019Q2 2019-04-12 14:11:33.126948
13 C_Quadrant II_S_H_2019Q2 2019-04-12 14:11:33.157803
14 C_Quadrant II_S_L_2019Q2 2019-04-12 14:11:33.177730
15 C_Quadrant II_S_M_2019Q2 2019-04-12 14:11:33.201022
16 C_Quadrant II_T_H_2019Q2 2019-04-12 14:11:33.215923
17 C_Quadrant II_T_L_2019Q2 2019-04-12 14:11:33.225210
18 C_Quadrant II_T_M_2019Q2 2019-04-12 14:11:33.238495
19 C_Quadr

In [47]:
lapsed_trans=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed'],dtype=str).drop_duplicates() # Doesn't go to score at all, so no need to read all columns
lapsed_trans['lapsed_trans']=True

lapsed_master=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     nrows = samplerows,dtype = 'str',sep = '|',
                       usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'])

lapsed_master=lapsed_master.drop_duplicates("customer_id_hashed")
print(lapsed_master.shape)

lapsed_master=pd.merge(lapsed_master,lapsed_trans,on="customer_id_hashed",how="outer")
print(lapsed_master.shape)

(1873409, 3)
(1873954, 4)


In [48]:
lapsed_master=lapsed_master[~lapsed_master['customer_id_hashed'].isin(test_all_df['customer_id_hashed'])]
lapsed_master=lapsed_master[~lapsed_master['customer_id_hashed'].isin(control_all_df['customer_id_hashed'])]
lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(test_all_df['email_address_hash'])]
lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(control_all_df['email_address_hash'])]

lapsed_master.shape

(832725, 4)

In [49]:
lapsed_master=lapsed_master[['customer_id_hashed','email_address_hash']]
lapsed_master['segment']="WalkingDead_2019Q2"

In [56]:
lapsed_master.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_2019-04-12/by_group/WalkingDead_Group_before_20160626.csv",index=False)

In [50]:
summary_test=test_all_df.groupby('segment_2019Q2')['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count","segment_2019Q2":"segment"})
summary_control=control_all_df.groupby('segment_2019Q2')['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count","segment_2019Q2":"segment"})
summary_WD=lapsed_master.groupby('segment')['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})

summary_overll=summary_test.append(summary_control).append(summary_WD)
summary_overll.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_2019-04-12/test_control_groups_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [51]:
summary_overll.head(2)

,segment,id_count
0,T_Quadrant III_P_H_2019Q2,1139050
1,T_Quadrant III_P_L_2019Q2,1524224


In [52]:
control_all_df.head(2)

,index,frmindex,customer_id_hashed,RFM,recency,transactions,sales,active,email_address_hash,customer_zip_code,zipcodegroup,location_id,Quadrant,HML_Group,segment_2019Q2
8,8,D01,b488318d6a842d9254a9ada4c9297f5ed1af3665876cd4...,111.0,1.0,350,9633.09,active,2e3f4dbfce053216b624f9e488f5319ab1fe28dc903a59...,70072,P,1180,Quadrant III,H,C_Quadrant III_P_H_2019Q2
108,108,D01,fe5e360edd31f9cec0dd3f82d2c0d80c5dea6c68af78f4...,111.0,1.0,95,534.98,active,02c1bde6666087a502f00f3d586f5d7ffe45bddd781c12...,75052,P,4622,Quadrant III,H,C_Quadrant III_P_H_2019Q2


In [53]:
test_all_df.groupby(['frmindex'])['customer_id_hashed'].count()

frmindex
D01    2357787
D02    2357672
D03    2357774
D04    2357796
D05    2357978
D06    2357709
D07    2357583
D08    2357301
D09    2357996
D10    2357968
Name: customer_id_hashed, dtype: int64

In [54]:
control_all_df.groupby(['frmindex'])['customer_id_hashed'].count()

frmindex
D01    49970
D02    50085
D03    49983
D04    49961
D05    49779
D06    50048
D07    50174
D08    50456
D09    49761
D10    49784
Name: customer_id_hashed, dtype: int64